Conver COCO to Yolo

In [3]:
from pycocotools.coco import COCO
import os
import shutil

def convert_coco_to_yolo(coco_json_path, output_img_dir, output_lbl_dir):
    os.makedirs(output_lbl_dir, exist_ok=True)

    coco = COCO(coco_json_path)
    image_ids = coco.getImgIds()
    category_ids = coco.getCatIds()
    categories = coco.loadCats(category_ids)
    category_mapping = {cat['id']: idx for idx, cat in enumerate(categories)}

    for img_id in image_ids:
        img_info = coco.loadImgs(img_id)[0]
        file_name = img_info['file_name']
        width = img_info['width']
        height = img_info['height']

        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)

        label_path = os.path.join(output_lbl_dir, os.path.splitext(file_name)[0] + '.txt')
        with open(label_path, 'w') as f:
            for ann in anns:
                if ann['iscrowd']: continue
                x, y, w, h = ann['bbox']
                class_id = category_mapping[ann['category_id']]
                x_center = (x + w / 2) / width
                y_center = (y + h / 2) / height
                w /= width
                h /= height
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

    print(f"Converted {len(image_ids)} images from COCO to YOLO format.")

# Example use:
convert_coco_to_yolo(
    coco_json_path='data/val/annotations_new.json',
    output_img_dir='data/val/images_new',
    output_lbl_dir='data/val/labels_new'
)


loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Converted 946 images from COCO to YOLO format.


Cek Jumlah Kelas

In [7]:
import json

# Ganti dengan path file JSON kamu
with open("data/val/annotations_new.json", "r") as f:
    data = json.load(f)

# Ambil dan urutkan berdasarkan ID kategori
categories = sorted(data["categories"], key=lambda x: x["id"])

# Ambil nama-nama kelas
class_names = [cat["name"] for cat in categories]

# Tampilkan dalam format YAML untuk YOLO
print("names:", class_names)
print("Jumlah Kelas", len(sorted(class_names)))


names: ['beetroot-steamed-without-addition-of-salt', 'green_bean_steamed_without_addition_of_salt', 'watermelon_fresh', 'tuna_in_oil_drained', 'sweet_potato', 'country_fries', 'potato_gnocchi', 'chips_french_fries', 'rosti_n_s', 'vegetable_mix_peas_and_carrots', 'coleslaw_chopped_without_sauce', 'ratatouille', 'mixed_salad_chopped_without_sauce', 'leaf_spinach', 'witloof_chicory', 'artichoke', 'aubergine_eggplant', 'avocado', 'french_beans', 'cucumber_pickled_ch', 'cucumber', 'capers', 'pumpkin', 'tomato', 'red_radish', 'beetroot_raw', 'chinese_cabbage', 'sauerkraut', 'white_cabbage', 'peas', 'corn', 'spring_onion_scallion', 'garlic', 'swiss_chard', 'white_asparagus', 'shoots_n_s', 'alfa_sprouts', 'beans_kidney', 'pineapple', 'apple', 'pomegranate', 'apricots', 'banana', 'berries_n_s', 'dates', 'strawberries', 'figs', 'grapefruit_pomelo', 'blueberries', 'raspberries', 'kaki', 'cherries', 'kiwi', 'mandarine', 'mango', 'sugar_melon_galia_honeydew_cantaloupe', 'nectarine', 'orange', 'plum

In [9]:
import wandb
from wandb.integration.ultralytics import add_wandb_callback
from ultralytics import YOLO

# Initialize WandB with project and experiment name
wandb.init(project="yolov9-newannotation")

# Load YOLOv9 model
model = YOLO("yolov9s.pt")
model_path = "YoloV9NewAnnotation.pt"

add_wandb_callback(model, enable_model_checkpointing=True)

# Train model while logging to WandB
results = model.train(
    data="data/dataset2.yaml",
    epochs=300,
    batch=2,
    imgsz=640,
    #lr0=0.0025,  # Initial Learning Rate
    #lrf=0.01,  # Cosine learning rate decay factor (bisa coba 0.1 - 0.5)
    #optimizer="Adam",  # Bisa juga coba Adam atau AdamW
    # hsv_h=0.1, hsv_s=0.7, hsv_v=0.4, flipud=0.5, fliplr=0.5,
    save_period=2,
    patience=50,
    # weight_decay=0.0005,
    #cos_lr=False,
    #amp=True,
    device="cuda",
    project="yolov9-trainingnew",
    name="new-experiment",
    verbose=True
)


model.save(model_path)

# Log performance metrics
wandb.log({"model": wandb.Artifact(model_path, type="model")})

wandb.finish()


New https://pypi.org/project/ultralytics/8.3.108 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.78 🚀 Python-3.9.19 torch-1.10.1 CUDA:0 (NVIDIA GeForce RTX 3060, 12029MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov9s.pt, data=data/dataset2.yaml, epochs=300, time=None, patience=50, batch=2, imgsz=640, save=True, save_period=2, cache=False, device=cuda, workers=8, project=yolov9-trainingnew, name=new-experiment, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agno

train: Scanning /home/bdsrc/mahmud/data/train/labels.cache... 54392 images, 0 backgrounds, 0 corrupt: 100%|██████████| 54392/54392 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/bdsrc/mahmud/data/train/images/006615.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/bdsrc/mahmud/data/train/images/006969.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/bdsrc/mahmud/data/train/images/007050.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/bdsrc/mahmud/data/train/images/007065.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /home/bdsrc/mahmud/data/train/images/007315.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/bdsrc/mahmud/data/train/images/007528.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/bdsrc/mahmud/data/train/images/007532.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /home/bdsrc/mahmud/data/train/images/007742.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/bdsrc/mahmud/data/train/images/007759.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/bdsrc/mahmud/data/train/images/007829.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /home/bdsrc/mahmud/data/train/images/00785

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/bdsrc/mahmud/data/val/labels.cache... 946 images, 0 backgrounds, 0 corrupt: 100%|██████████| 946/946 [00:00<?, ?it/s]

...

                   all        946       1704       0.61      0.596      0.661      0.549
beetroot-steamed-without-addition-of-salt          5          5      0.453        0.8      0.775       0.63
       bread_wholemeal         36         36      0.504      0.722      0.632      0.573
                   jam         21         21      0.769      0.857      0.899      0.673
                 water         78         80       0.82      0.925      0.924      0.791
                 bread         21         21      0.748      0.566        0.6      0.502
                banana          7          7      0.483          1       0.93      0.668
           soft_cheese          6          6      0.151      0.167      0.323      0.282
               ham_raw          6          6      0.746      0.492      0.812      0.655
           hard_cheese         12         12      0.628       0.75      0.661      0.526
        cottage_cheese          4          4      0.646          1      0.895      0.706
  

Generating Visualizations for batch-1/1:   0%|          | 0/4 [00:00<?, ?it/s]

Speed: 0.3ms preprocess, 5.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to yolov9-trainingnew/new-experiment


lr/pg0,████▇▇▇▆▆▆▆▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
lr/pg1,█████▇▇▇▇▇▇▆▆▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▁▁▁▁
lr/pg2,▃███▇▇▇▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▁▁▁
metrics/mAP50(B),▁▂▂▃▃▃▃▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇██
metrics/mAP50-95(B),▁▂▂▂▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇███
metrics/precision(B),▇▄▃▃▄▂▃▂▂▂▁▁▁▁▂▂▂▃▂▂▂▃▄▃▂▃▃▃▂▃▄▄▅▆▆████▇
metrics/recall(B),▁▁▂▃▃▄▄▅▅▅▅▅▅▅▆▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇▆▇▇▇▇▇▇███
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▃▃▃▃▁


Error: You must call wandb.init() before wandb.log()